In [1]:
import sys
import os
import re
import pandas as pd

from horsePlayground import parseFullDay, bulkParse

from genInfoFns import parseGenInfo
from horseInfoFns import parseHorseInfo
from timesInfoFns import parseTimeInfo
from betInfoFns import parseBetInfo
from runlineInfoFns import parseRunlineInfo
from endInfoFns import parseEndInfo


with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.112.txt') as file:
    test1 = file.readlines()
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.148.txt') as file:
    test2 = file.readlines()
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-14T221922.451.txt') as file:
    test3 = file.readlines()

In [2]:
directory = '/home/karisch/programming/projects/horseData/charts/chartsTest'
jack = bulkParse(directory)

1
2
3
4
5
6
 3 Champagne Chuck --- --- --- --- --- ---



AttributeError: 'NoneType' object has no attribute 'group'

In [12]:
line = ' 3 Champagne Chuck 5B --- --- --- --- ---'

In [13]:
re.search(r'^ (\d?\d[ABC]?) [^0-9]+ \d?\d[ABC]?( ---)+$', line)

<re.Match object; span=(0, 41), match=' 3 Champagne Chuck 5B --- --- --- --- ---'>